In [1]:
import torch
import torch.nn as nn
import random
import numpy as np
import pandas as pd
import json
import cv2
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from functools import partial
from dataclasses import dataclass
from collections import OrderedDict
import torchvision
from IPython.display import clear_output
import matplotlib.pyplot as plt
import tqdm
from PIL import ImageFilter, Image
import torchvision.models as models
import pretrainedmodels as ptmodels
import torch.nn as nn
import torch.nn.functional as F
import math
#import torch.utils.model_zoo as model_zoo
from torch.nn import init

from network.meso import Meso4, MesoInception4
from network.models import model_selection

#from ff import model_selection
from pipeline.metrics import accuracy_b, log_loss
from pipeline.model_methods import validate_img, train_img, validate_vid, train_vid
from pipeline.data_loaders import load_img_dataset, load_img_val_dataset, strong_aug
from pipeline.image_extracting import extract_faces, FastMTCNN

from pipeline.metrics import accuracy_sigmoid, accuracy_sigmoid_mean, log_loss_sigmoid, log_loss_b, accuracy_b_mean
from pipeline.model_methods import validate_vid_bf, train_vid_bf
from pipeline.image_extracting import extract_faces, FastMTCNN, InceptionResnetV1, extract_faces_dlib, MTCNN
from pipeline.blazeface import BlazeFace


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
PATH = "model.h5"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
fast_mtcnn = FastMTCNN(
    stride=4,
    resize=1,
    margin=14,
    factor=0.9,
    #min_face_size=60,
    #keep_all=True,
    select_largest=True,
    device=device
)
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
facedet = BlazeFace().to(device)
facedet.load_weights("pipeline/blazeface.pth")
facedet.load_anchors("pipeline/anchors.npy")
_ = facedet.train(False)

from pipeline.helpers.read_video_1 import VideoReader
from pipeline.helpers.face_extract_1 import FaceExtractor

frames_per_video = 17

video_reader = VideoReader()
video_read_fn = lambda x: video_reader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn, facedet)

In [4]:
y_train = pd.read_csv(r'D:\Machine Learning\deepfake-detection\data\metadata.csv')
len(y_train[y_train.label == 1]), len(y_train[y_train.label == 0])

(100000, 19154)

In [5]:
#%%script false

batch_size = 16
normalize = torchvision.transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))

x_train = load_img_dataset(r'D:\Machine Learning\deepfake-detection\data\frames', batch_size, resize=224, normalize=normalize)
#x_val = load_img_val_dataset('data/val/', 10, resize=224, normalize=normalize)

x_train_vid = r'D:\Machine Learning\deepfake-detection\data\train_set'
x_val_vid = r'D:\Machine Learning\deepfake-detection\data\min_val'
x_test_vid = r'D:\Machine Learning\deepfake-detection\data\cross_val'

In [6]:
class MyResNeXt(models.resnet.ResNet):
    def __init__(self, training=True):
        super(MyResNeXt, self).__init__(block=models.resnet.Bottleneck,
                                        layers=[3, 4, 6, 3], 
                                        groups=32, 
                                        width_per_group=4)
        #self.fc = nn.Linear(2048, 1)

In [7]:
model = MyResNeXt()#models.resnext50_32x4d(pretrained=True) models.resnext101_32x8d(pretrained=True) MyResNeXt()

In [8]:
#%%script false

model.load_state_dict(torch.load("pretrained/resnext50_32x4d-7cdf4587.pth", map_location=device))
#model.load_state_dict(torch.load("0.261764258146286 0.9066666666666666.pth", map_location=device))
_ = model.eval()

In [9]:
model.fc = nn.Linear(2048, 1)

In [10]:
model = model.to(device)

In [11]:
loss =  torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.) 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)
#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda epoch: epoch // 30, lambda epoch: 0.95 ** epoch], last_epoch=-1)

In [12]:
i = 0
for _ in model.children():
    i += 1
print(i)

10


In [13]:
#%%script false
count = 0
for child in model.children():  
    count+=1
    if count < 8:    
        for param in child.parameters():
            param.requires_grad = False

In [14]:
[i[0] for i in model.named_parameters() if i[1].requires_grad]

['layer4.0.conv1.weight',
 'layer4.0.bn1.weight',
 'layer4.0.bn1.bias',
 'layer4.0.conv2.weight',
 'layer4.0.bn2.weight',
 'layer4.0.bn2.bias',
 'layer4.0.conv3.weight',
 'layer4.0.bn3.weight',
 'layer4.0.bn3.bias',
 'layer4.0.downsample.0.weight',
 'layer4.0.downsample.1.weight',
 'layer4.0.downsample.1.bias',
 'layer4.1.conv1.weight',
 'layer4.1.bn1.weight',
 'layer4.1.bn1.bias',
 'layer4.1.conv2.weight',
 'layer4.1.bn2.weight',
 'layer4.1.bn2.bias',
 'layer4.1.conv3.weight',
 'layer4.1.bn3.weight',
 'layer4.1.bn3.bias',
 'layer4.2.conv1.weight',
 'layer4.2.bn1.weight',
 'layer4.2.bn1.bias',
 'layer4.2.conv2.weight',
 'layer4.2.bn2.weight',
 'layer4.2.bn2.bias',
 'layer4.2.conv3.weight',
 'layer4.2.bn3.weight',
 'layer4.2.bn3.bias',
 'fc.weight',
 'fc.bias']

In [15]:
%%script false

# default - Validation: metrics  0.6033333333333334 loss:  0.6769222617149353

validate_vid_bf(model, x_val_vid, y_train, log_loss_sigmoid, accuracy_sigmoid_mean, device, 2, face_extractor, 
                fast_mtcnn=fast_mtcnn,
                print_results=True, 
                reverse=False)

Couldn't find program: 'false'


In [16]:
%%script false

validate_img(model, x_val, y_train, loss, accuracy_b, device, 10, 
             print_results=True, 
             inference=nn.Sigmoid(),
             checkpoint=None,
             reverse=False
            )

Couldn't find program: 'false'


In [17]:
0.3311 * 100 / 71

0.4663380281690141

In [ ]:
#%%script false
#Validation: metrics  0.9066666666666666 loss:  0.261764258146286 2 0.001
#Validation: metrics  0.9433333333333334 loss:  0.2010902613401413 14 0.01
#Validation: metrics  0.9433333333333334 loss:  0.19530311226844788 19 0.01
#Validation: metrics  0.9466666666666667 loss:  0.1870456337928772 23 0.01
#57% /58.5% - 0.656% (0.99344%)
metric, lost = train_img(model, loss, optimizer, scheduler, x_train, x_val_vid, y_train, accuracy_b, device, [log_loss_sigmoid, accuracy_sigmoid_mean], face_extractor, 
                          epochs=50, 
                       batch_size=batch_size, 
                       del_net=False, 
                       useInference=True, 
                       inference=nn.Sigmoid(), 
                       useScheduler=True,
                       checkpoint=[0.2, 0.99],
                         reverse=False
                      )


Validation: metrics  0.88 loss:  0.3483356833457947
Train: metrics  0.7347698389241909 loss  0.5182076307027118



Validation: metrics  0.8766666666666667 loss:  0.2925425171852112
Train: metrics  0.7990431505837151 loss  0.4217961075824033



Validation: metrics  0.9 loss:  0.276156485080719
Train: metrics  0.8246915176592287 loss  0.37517181948076167



Validation: metrics  0.9 loss:  0.2565360963344574
Train: metrics  0.8391089108910891 loss  0.34827239780041463



Validation: metrics  0.8966666666666666 loss:  0.2534264326095581
Train: metrics  0.8485296290823112 loss  0.32950318871263884


...


Validation: metrics  0.91 loss:  0.2152564823627472
Train: metrics  0.8973695877050392 loss  0.23357845721588813



Validation: metrics  0.9266666666666666 loss:  0.2012469470500946
Train: metrics  0.8979791635880006 loss  0.23175167239548974



Validation: metrics  0.9466666666666667 loss:  0.1870456337928772
Train: metrics  0.8986441554603222 loss  0.22970275754582187



Validation: metrics  0.94 loss:  0.2027776837348938
Train: metrics  0.9004821191074331 loss  0.2279701417274041



Validation: metrics  0.9233333333333333 loss:  0.19244322180747986
Train: metrics  0.9009439190187676 loss  0.22675247061050777


In [ ]:
from operator import itemgetter
min_lost = max(enumerate(lost), key=itemgetter(1))[0] 
min_metrics = min(enumerate(metric), key=itemgetter(1))[0] 
print("Loss optim:", lost[min_lost], min_lost + 1)
print("Metrics optim:", metric[min_metrics], min_metrics + 1)

In [ ]:
plt.plot(metric, label='model')
plt.legend()
plt.title('Validation metrics')

In [ ]:
plt.plot(lost, label='model')
plt.legend()
plt.title('Validation Loss')

In [ ]:
'''model = torch.load(PATH)
model = model.to(device)
model.eval()'''

In [ ]:
validate_img(model, x_val, y_train, loss, log_loss, device, batch_size, print_results=True, inference=nn.Softmax(dim=1))

In [ ]:
validate_img(model, x_val, y_train, loss, log_loss, device, batch_size, show_results=True, inference=nn.Softmax(dim=1))

In [ ]:
validate_img(model, x_val, y_train, loss, log_loss, device, batch_size, show_graphic=True, inference=nn.Softmax(dim=1))

In [ ]:
#torch.save(model.state_dict(), '0.74 0.5604.pth')